In [2]:
import pandas as pd

github_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv"
url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [3]:
df = df.iloc[:300]

In [4]:
from sentence_transformers import SentenceTransformer

model_name = "multi-qa-mpnet-base-dot-v1"
embedding_model = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
answer_llm = df.iloc[0].answer_llm

In [12]:
embeddings = embedding_model.encode(answer_llm)

# Get the first value of the resulting vector
first_value = embeddings[0]
print(first_value)

-0.42244655


In [16]:
import numpy as np

df['embeddings_answer_llm'] = df['answer_llm'].apply(lambda x: embedding_model.encode(x))
df['embeddings_answer_orig'] = df['answer_orig'].apply(lambda x: embedding_model.encode(x))

evaluations = []

for i in range(len(df)):
        dot_product = np.dot(df.iloc[i]['embeddings_answer_llm'], df.iloc[i]['embeddings_answer_orig'])
        evaluations.append(dot_product)

percentile_75 = np.percentile(evaluations, 75)
print(percentile_75)

31.67430877685547


In [21]:
def encode_n_normalize(text):
    v = embedding_model.encode(text)
    norm = np.sqrt((v * v).sum())
    v_norm = v / norm
    return v_norm

df['norm_embeddings_answer_llm'] = df['answer_llm'].apply(lambda x: encode_n_normalize(x))
df['norm_embeddings_answer_orig'] = df['answer_orig'].apply(lambda x: encode_n_normalize(x))


norm_evaluations = []

for i in range(len(df)):
        dot_product = np.dot(df.iloc[i]['norm_embeddings_answer_llm'], df.iloc[i]['norm_embeddings_answer_orig'])
        norm_evaluations.append(dot_product)

percentile_75 = np.percentile(norm_evaluations, 75)
print(percentile_75)

0.8362348973751068


In [27]:
r = df.iloc[10]

In [28]:
from rouge import Rouge
rouge_scorer = Rouge()

scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]

In [29]:
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

In [32]:
f_scores = [
    scores['rouge-1']['f'],
    scores['rouge-2']['f'],
    scores['rouge-l']['f']
]

average_f_score = sum(f_scores) / len(f_scores)
print("Average F-score:", average_f_score)


Average F-score: 0.35490034990035496


In [36]:
def calculate_rouge(text1, text2):
    return rouge_scorer.get_scores(text1, text2)[0]

df['rouge_scores'] = df.apply(lambda row: calculate_rouge(row['answer_llm'], row['answer_orig']), axis=1)


In [37]:
df['rouge_scores']

0      {'rouge-1': {'r': 0.061224489795918366, 'p': 0...
1      {'rouge-1': {'r': 0.08163265306122448, 'p': 0....
2      {'rouge-1': {'r': 0.32653061224489793, 'p': 0....
3      {'rouge-1': {'r': 0.16326530612244897, 'p': 0....
4      {'rouge-1': {'r': 0.2653061224489796, 'p': 0.0...
                             ...                        
295    {'rouge-1': {'r': 0.6428571428571429, 'p': 0.6...
296    {'rouge-1': {'r': 0.6428571428571429, 'p': 0.5...
297    {'rouge-1': {'r': 0.6607142857142857, 'p': 0.6...
298    {'rouge-1': {'r': 0.2857142857142857, 'p': 0.3...
299    {'rouge-1': {'r': 0.125, 'p': 0.31818181818181...
Name: rouge_scores, Length: 300, dtype: object

In [41]:
df['rouge-2'] = df['rouge_scores'].apply(lambda x: x['rouge-2'])
df['rouge-2-f'] = df['rouge-2'].apply(lambda x: x['f'])

In [43]:
df['rouge-2-f'].mean()

0.20696501983423318